## Rating csv


In [1]:
import pandas as pd
import datetime
import numpy as  np
import torch
from tqdm import tqdm
from scipy.sparse import csr_matrix
np.random.seed(1337)
with open('./kaggle/rating_train.csv', 'r') as f:
    ls = f.readlines()[1:]
u_map = {}

dates = []
foods = []
users = []
    


with tqdm(total=len(ls)) as pbar:
    for l in ls:
        date_str, user, food = l.strip().split(',')
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
        user, food = int(user), int(food)
        if user not in u_map:
            u_map[user] = []
        u_map[user].append( (date, food) )
        
        dates.append(date)
        users.append(user)
        foods.append(food)
        pbar.update(1)
        

user_map = {u:i for i, u in enumerate(set(users))}        
food_map = {f:i for i, f in enumerate(set(foods))}


# for ranking sparse matrix
rows = [user_map[u] for u in users]
cols = [food_map[f] for f in foods]
R = csr_matrix((np.ones([len(rows), ]), (rows, cols)), shape=(len(user_map), len(food_map)))

pos_count = np.array(np.sum(R, axis=0)).flatten()
neg_count = len(ls) - pos_count


class_weight =  1. / 2.*neg_count
pos_weight = neg_count / pos_count
pos_weight = np.ones_like(pos_count) *20.
print R.shape


pos_count = torch.FloatTensor(pos_count).cuda()
pos_count.requires_grad_(False)
neg_count = torch.FloatTensor(neg_count).cuda()
neg_count.requires_grad_(False)

print neg_count.shape
print pos_count.shape



100%|██████████| 2681494/2681494 [00:20<00:00, 131996.02it/s]


(2608, 5532)
torch.Size([5532])
torch.Size([5532])


#### prepare sample generator


In [2]:
from constants import MAX_SEQ_LEN


idx = np.random.permutation(len(u_map))
val_num = len(u_map)//10
train_idx, val_idx = idx[val_num:], idx[:val_num]
train_u_map = {k:u_map[k] for k in u_map.keys()[val_num:]}
val_u_map = {k:u_map[k] for k in u_map.keys()[:val_num]}
def batch_boostrap_generator(batch_size, u_map, food_map, max_history_len):
    G = boostrap_generator(u_map, food_map, max_history_len)
    while True:
        X = []
        pad_masks = []
        for i in range(batch_size):
            x, x_len = next(G)
            X.append(np.expand_dims(x, axis=0))
            pad_mask = np.zeros_like(x)
            for idx in range(x_len):
                pad_mask[idx, :] = (max_history_len-x_len+idx+1)*0.3
            pad_masks.append(np.expand_dims(pad_mask, axis=0))
        yield np.vstack(X), np.vstack(pad_masks)
def boostrap_generator(u_map, food_map, max_history_len):
    while True:
        keys = u_map.keys()
        for user_idx in np.random.permutation(len(u_map)):
            user = keys[user_idx]
            X = np.zeros([max_history_len, len(food_map)])
            history = u_map[user]
            ds = np.array([d for d,f in history])
            fs = np.array([f for d,f in history])
            sorted_idx = np.argsort(ds)
            ds = ds[sorted_idx]
            fd = fs[sorted_idx]
            
            date_idx = 0
            now_date = ds[0]
            for food, date in zip(fs,ds):
                if date != now_date:
                    date_idx+=1
                    now_date = date
                X[date_idx, food_map[food]] = 1
            x_len = date_idx+1
            yield X, x_len
            
    

G = batch_boostrap_generator(32, train_u_map, food_map, max_history_len=MAX_SEQ_LEN)
# val_G = batch_boostrap_generator(32//2, val_u_map, food_map, max_history_len=MAX_SEQ_LEN)

x, pad_mask = next(G)
print x.shape, pad_mask.shape
# x, pad_mask = next(val_G)
# print x.shape, pad_mask.shape

# G2 = boostrap_generator(train_u_map, food_map, max_history_len=MAX_SEQ_LEN)
# x, x_len = next(G2)
# print x.shape
print pad_mask[3,:,3]

(32, 165, 5532) (32, 165, 5532)
[21.3 21.6 21.9 22.2 22.5 22.8 23.1 23.4 23.7 24.  24.3 24.6 24.9 25.2
 25.5 25.8 26.1 26.4 26.7 27.  27.3 27.6 27.9 28.2 28.5 28.8 29.1 29.4
 29.7 30.  30.3 30.6 30.9 31.2 31.5 31.8 32.1 32.4 32.7 33.  33.3 33.6
 33.9 34.2 34.5 34.8 35.1 35.4 35.7 36.  36.3 36.6 36.9 37.2 37.5 37.8
 38.1 38.4 38.7 39.  39.3 39.6 39.9 40.2 40.5 40.8 41.1 41.4 41.7 42.
 42.3 42.6 42.9 43.2 43.5 43.8 44.1 44.4 44.7 45.  45.3 45.6 45.9 46.2
 46.5 46.8 47.1 47.4 47.7 48.  48.3 48.6 48.9 49.2 49.5  0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]


### Transformer model with one-for-all BCE loss

In [3]:
# import imp
# from os.path import expanduser
# home = expanduser("~")
# Transformer = imp.load_source('Transformer', '%s/git/grandchallenge/ASR/Transformer.py' % home)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
# construct neuron network

def scaled_dot_attention(Q, K, V, mask):
    assert Q.size()[-1] == K.size()[-1]
    assert len(Q.size()) == 3 and len(K.size()) == 3 and len(V.size()) == 3
    dk = torch.tensor(K.size()[-1], dtype=torch.float32, requires_grad=False).cuda()
    out = torch.matmul(Q,K.permute(0,2,1)) / torch.sqrt(dk) 
    if mask is not None:
        out.masked_fill_(mask, -float('inf'))
    return torch.matmul(F.softmax(out, dim=-1), V)

def positional_encoding(d_model, pos):
    assert d_model % 2 == 0
    pos = torch.tensor(pos, dtype=torch.float32, requires_grad=False)
    pe = torch.zeros([1,d_model], dtype=torch.float32, requires_grad=False)
    for i in range(D_MODEL//2):
        a = torch.tensor(10000, dtype=torch.float32, requires_grad=False)
        b = torch.tensor(2.*i/float(D_MODEL), dtype=torch.float32, requires_grad=False)
        c = pos / torch.pow(a, b)
        pe[0, 2*i] = torch.sin(c)
        pe[0, 2*i+1] = torch.cos(c)
    return pe
                            
class Transformer_v2(nn.Module):

    def __init__(self, layer_num, dk, dv, dm, h, p_drop, d_ff, use_mask, use_cuda=True, posi_cache_length=200):
        super(Transformer_v2, self).__init__()
#         for construct cache positional encoding matrix.
        self.d_model = dm
        self.use_cuda = use_cuda
        
        self.decoder = Stack_Decoder(layer_num, dk, dv, dm, h, p_drop, d_ff, use_mask)
        self.emb_drop = nn.Dropout(p_drop)
        self.init_pos_mat(posi_cache_length)

    def forward(self, Q):
    
        
#         decoder
        batch, Q_len, d = Q.size()
        
        try:
            Q = Q + self.get_pos_mat(Q_len)
        except RuntimeError, e:
            if e.message == 'TensorIterator expected type torch.cuda.FloatTensor but got torch.FloatTensor':
                if Q.is_cuda != self.get_pos_mat(K_len).is_cuda:
                    print('Make sure cache positional matrix is same type of tensor with input, both cuda tensor or not.\nBy setting argument use_cuda=True to set cache positional encoding matrix as a cuda tensor.')
            raise
        
        Q = self.emb_drop(Q)
        
        de_out = self.decoder(Q)
        return de_out
    
#     To speed up the positional encoding by construct an cache matrix. 
    def init_pos_mat(self, cache_length):
        print('init postional matrix with length : %d ' % cache_length)
        self.positional_matrix = torch.cat([positional_encoding(self.d_model, i) for i in range(0,cache_length)], dim=0)
        self.positional_matrix.requires_grad = False
        if self.use_cuda:
            self.positional_matrix = self.positional_matrix.cuda()
            
        
    def get_pos_mat(self, length):
        if length > self.positional_matrix.shape[0]:
            print('input sequence length reach positional matrix maximum length. %d ' % length)
            ret = torch.cat([positional_encoding(self.d_model, i) for i in range(length)], dim=0)
            ret.requires_grad = False
            print('Increase positional matrix maximum length. %d ' % length)
            self.positional_matrix = ret
            if self.use_cuda:
                self.positional_matrix = self.positional_matrix.cuda()
            return ret
        else:
            return self.positional_matrix[:length]
        

    
    

class Stack_Decoder(nn.Module):
    """
    Stacked Decoder
    """
    def __init__(self, layer_num, dk, dv, dm, h, p_drop, d_ff, use_mask):
        super(Stack_Decoder, self).__init__()
        self.decoders = nn.ModuleList([Decoder(dk, dv, dm, h, p_drop, d_ff, use_mask) for i in range(layer_num)])
        
        
    def forward(self, Q):
        # ModuleList can act as an iterable, or be indexed using ints
        for lay in self.decoders:
            Q = lay(Q)
        return Q           

class Decoder(nn.Module):
    def __init__(self, dk, dv, dm, h, p_drop, d_ff, use_mask):
        super(Decoder, self).__init__()
        self.use_mask = use_mask
        
#         query attention residual block
        self.Q_attention_lay = Multi_Head_attention_layer(dk, dv, dm, h)
        self.Q_attention_norm_lay = nn.LayerNorm([dm, ])
        self.Q_att_drop = nn.Dropout(p_drop)

#         feed forward residual block
        self.fcn = PositionwiseFeedForward(dm, d_ff)
        self.ff_norm_lay = nn.LayerNorm([dm, ])
        self.linear_drop = nn.Dropout(p_drop)
        

    def forward(self, Q):
        if self.use_mask:
            batch, Q_len, d = Q.size()
            mask = self.mask_matrix(batch, Q_len)
        else:
            mask = None
#         query attention
        Q_attention_out = self.Q_attention_lay(Q, Q, Q, mask=mask)
        Q_attention_out = self.Q_att_drop(Q_attention_out)
        Q_att_out = self.Q_attention_norm_lay(Q + Q_attention_out)
        
#         feed forward
        linear_out = self.fcn(Q_att_out)
        out = self.ff_norm_lay(Q_att_out + linear_out)
        return out
    def mask_matrix(self, batch, Q_len):
#         ByteTensor
        mask = torch.zeros([1, Q_len, Q_len], dtype=torch.uint8, requires_grad=False)
        for i in range(Q_len):
            mask[0,i,i+1:] = 1
        return mask.repeat(batch,1, 1).cuda()


class Multi_Head_attention_layer(nn.Module):
    def __init__(self, dk, dv, dm, h):
        super(Multi_Head_attention_layer, self).__init__()
        self.Q_linears = nn.ModuleList([nn.Linear(dm, dk) for i in range(h)])
        self.K_linears = nn.ModuleList([nn.Linear(dm, dk) for i in range(h)])
        self.V_linears = nn.ModuleList([nn.Linear(dm, dv) for i in range(h)])
        self.output_linear = nn.Linear(h*dv, dm)
                            

    def forward(self, Q_input, K_input, V_input, mask):
        buf = []
        for Q_linear, K_linear, V_linear in zip(self.Q_linears, self.K_linears, self.V_linears):
            Q = Q_linear(Q_input)
            K = K_linear(K_input)
            V = V_linear(V_input)
            buf.append(scaled_dot_attention(Q, K, V, mask))
        
        buf = torch.cat(buf,dim=-1)
        out = self.output_linear(buf)
        
        return out      
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.cnn1 = nn.Conv1d(d_model, d_ff, 1)
        self.cnn2 = nn.Conv1d(d_ff, d_model, 1)
                            

    def forward(self, x):
        bat,seq_len,d = x.size()
        x = x.permute(0,2,1)
        x = self.cnn1(x)
        x = F.relu(x)
        x = self.cnn2(x)
        x = x.permute(0,2,1)
        
        return x      
    

    
    
# Transformer paper baseline hyper-parameters
STACKED_NUM = 2
H = 4
D_MODEL = 128
DK = DV = D_MODEL//H
P_DROP = 0.05
D_FF = D_MODEL*4




    
# bat = 3
# Q = torch.rand([bat, 13, D_MODEL]).cuda()
# model = Transformer_v2(STACKED_NUM, DK, DV, D_MODEL, H, P_DROP, D_FF, use_mask=True, use_cuda=True).cuda()
# o = model(Q)
# print(o.size())

# Q = torch.rand([bat, 47, D_MODEL]).cuda()
# o = model(Q)
# print(o.size())
# # # print o
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(count_parameters(model))



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F### Transformer with ALS embedding Training
# import Transformer/

import numpy as np
from constants import FOOD_NUM, USER_NUM
class Net(nn.Module):

    def __init__(self, dm, p_drop):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p_drop)
        self.food_emb = Food_embedding(FOOD_NUM, dm, 1, p_drop)
        self.transformer = Transformer_v2(STACKED_NUM, DK, DV, D_MODEL, H, P_DROP, D_FF, use_mask=True, use_cuda=True).cuda()

        self.output_linear = nn.Linear(dm, FOOD_NUM)

    def forward(self, history):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        
        x = self.food_emb(history)
        batch, x_len, d = x.size()
        
        x = self.transformer(x)
        x = self.output_linear(x)
#         x = torch.sigmoid(x)
        return x
        
class Food_embedding(nn.Module):
    def __init__(self, c_in, dm, layer_num, p_drop, activation_fn=F.selu):
        super(Food_embedding, self).__init__()
        self.activation_fn = activation_fn
        self.drop = nn.Dropout(p_drop)
        assert layer_num >= 1
        self.first_linear = nn.Linear(c_in, dm)
        self.linears = nn.ModuleList([nn.Linear(dm, dm) for i in range(layer_num-1)])
        

    def forward(self, x):
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        x = self.first_linear(x)
        for lay in self.linears:
            x = self.activation_fn(lay(x))
            if lay != self.linears[-1]:
                x = self.drop(x)
        return x
    
batch = 7
dm = D_MODEL
Q = torch.rand([batch, 18, FOOD_NUM]).cuda()
model = Net(dm, 0.1).cuda()
o = model(Q)
# print t
print(o.size())
# print o

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))




init postional matrix with length : 200 
torch.Size([7, 18, 5532])
1818396


In [ ]:
from collections import deque
from tqdm import tqdm as tqdm

import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, precision, val_precision, recall, val_recall, f1, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc, precision, val_precision, recall, val_recall, f1)
    log_file_stream.write(log_text)
    if n_iter % 10 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)
def normal_acc(pred, label, pad_mask):
    label = label.type(torch.uint8)
    mask = (pad_mask != 0).type(torch.uint8)
#     
    acc = pred == label
    acc = acc.masked_select(mask)
    acc = torch.sum(acc).item() / float(torch.sum(mask).item())
#     
    TP = float(torch.sum(pred.masked_select(label)).item())
    TP_FP = torch.sum(pred.masked_select(mask)).item()
#     precision = TP / TP_FP if TP_FP != 0 else float('nan')
    precision = TP / TP_FP if TP_FP != 0. else 0.
    recall = TP / torch.sum(label).item()
# 
    f1 = 2.*precision*recall / (precision+recall) if (precision+recall) != 0  else 0
    
    return acc, precision, recall, f1
def rev_mask(m):
    out = torch.ones_like(m, dtype=torch.uint8, requires_grad=False)
    out.masked_fill_(m, 0)
    return out
        
acc_q = deque(maxlen=1000)
precision_q = deque(maxlen=1000)
recall_q = deque(maxlen=1000)
loss_q = deque(maxlen=1000)
f1_q = deque(maxlen=1000)

val_acc_q = deque(maxlen=10000)
val_loss_q = deque(maxlen=10000)
val_precision_q = deque(maxlen=10000)
val_recall_q = deque(maxlen=10000)
val_f1_q = deque(maxlen=1000)

t = time.time()
best_f1  = 0
best_loss = float('inf')

epochs = 100
batch_size = 16
G = batch_boostrap_generator(batch_size, train_u_map, food_map, max_history_len=MAX_SEQ_LEN)
val_G = batch_boostrap_generator(batch_size, val_u_map, food_map, max_history_len=MAX_SEQ_LEN)
criterion = nn.BCEWithLogitsLoss(reduction='none', pos_weight=torch.FloatTensor(pos_weight).cuda())
# criterion = nn.BCEWithLogitsLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
print 'start training.'
with open('log-heavy.txt', 'a') as f:
    with open('best-heavy.txt', 'w') as best_log:
        iters = 100000000
        with tqdm(total=iters) as pbar:
            for it in range(iters):
                optimizer.zero_grad()
                model.train()
                seq, pad_mask = next(G)
                seq = torch.FloatTensor(seq).cuda()
                pad_mask = torch.FloatTensor(pad_mask).cuda()
                seq.requires_grad_(False)
                pad_mask.requires_grad_(False)

                x = seq[:,:-1,:]
                y = seq[:,1:,:]

                output = model(x)
#                     a,b = foo(output, y)
#                     loss = normal_loss(criterion, output, y)

#                 loss = torch.sum(criterion(output, y) * pad_mask[:,1:, :]) / torch.sum(pad_mask[:,1:, :])
                loss = torch.sum(criterion(output, y) * pad_mask[:,1:, :]) 
                pred = output > 0.5
                label = y


                acc, precision, recall, f1 = normal_acc(pred, label, pad_mask[:,1:,:])
                acc_q.append(acc)
                precision_q.append(precision)
                recall_q.append(recall)
                f1_q.append(f1)
                loss.backward()

                optimizer.step()
                with torch.no_grad():
                    model.eval()
                    seq, pad_mask = next(val_G)
                    seq = torch.FloatTensor(seq).cuda()
                    pad_mask = torch.FloatTensor(pad_mask).cuda()
                    seq.requires_grad_(False)
                    pad_mask.requires_grad_(False)

                    x = seq[:,:-1,:]
                    y = seq[:,1:,:]
                    output = model(x)
#                     c,d = foo(output, y)
#                         val_loss = normal_loss(criterion, output, y)
#                         val_loss = criterion(output, y)
#                         pred = output > 0.5
#                     val_loss = torch.sum(criterion(output, y)* pad_mask[:,1:, :]) / torch.sum(pad_mask[:,1:, :])
                    val_loss = torch.sum(criterion(output, y)* pad_mask[:,1:, :]) 
    
                    pred = output > 0.5

                    label = y

                    val_acc, val_precision, val_recall, val_f1 = normal_acc(pred, label, pad_mask[:,1:,:])
                    val_acc_q.append(val_acc)
                    val_precision_q.append(val_precision)
                    val_recall_q.append(val_recall)
                    val_f1_q.append(val_f1)
                
                acc = np.mean(acc_q)
                precision = np.mean(precision_q)
                recall = np.mean(recall_q)
                f1 = np.mean(f1_q)
                
                val_acc = np.mean(val_acc_q)
                val_precision = np.mean(val_precision_q)
                val_recall = np.mean(val_recall_q)
                val_f1 = np.mean(val_f1_q)
                
#                     pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, loss : %.3f, val_loss : %.3f \t %.3f, %.3f, %.3f, %.3f' % (acc, val_acc, loss.item(), val_loss.item(), a,b,c,d), refresh=False)
                pbar.set_postfix_str('acc : %.3f, val_acc : %.3f, precision : %.3f, val_precision : %.3f, recall : %.3f, val_recall : %.3f, loss : %.3f, val_loss : %.3f, f1 : %.3f, val_f1 : %.3f' % (acc, val_acc, precision, val_precision, recall, val_recall, loss.item(), val_loss.item(), f1, val_f1), refresh=False)
                pbar.update(batch_size)
                dump_log(model, (it+1)*batch_size, loss, acc, precision, val_precision, recall, val_recall, val_loss, val_acc, f1, f, './tmp-heavy.pt')
                if val_f1 > best_f1 and it > 100:
                    torch.save(model, './best-heavy.pt')
                    best_f1 = val_f1
                    best_log.write('%d\t%.5f\n' % ((it+1)*batch_size, best_f1))

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

  0%|          | 0/100000000 [00:00<?, ?it/s]

start training.


  0%|          | 7184/100000000 [04:25<1140:22:21, 24.36it/s, acc : 0.997, val_acc : 0.997, precision : 0.189, val_precision : 0.195, recall : 0.279, val_recall : 0.286, loss : 23340080.000, val_loss : 21626264.000, f1 : 0.225, val_f1 : 0.231]

In [ ]:
print pred.shape, pos_count.shape
a = torch.FloatTensor(pos_count).cuda()
print a.shape
c = pred.type(torch.float)*a
print c.shape